In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import col, SparkContext, udf
from pyspark.sql.types import *
import numpy as np

In [2]:
appName = "Big Data ML"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
# spark = sqlContext.sparkSession.builder.getOrCreate()
spark = sqlContext.sparkSession.builder.getOrCreate()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\Users\Woodw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\python\lib\py4j-0.10.9.7-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\Woodw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\python\lib\py4j-0.10.9.7-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
mqtt_raw = spark.read.csv("train70_reduced.csv",header=True, inferSchema= True)
mqtt_test_raw = spark.read.csv("test30_reduced.csv",header=True, inferSchema= True)

# Rename columns to better formats
for column_name in mqtt_raw.columns:
    new_column_name = column_name.replace(".", "_")
    mqtt_raw = mqtt_raw.withColumnRenamed(column_name, new_column_name)

for column_name in mqtt_test_raw.columns:
    new_column_name = column_name.replace(".", "_")
    mqtt_test_raw = mqtt_test_raw.withColumnRenamed(column_name, new_column_name)

# Drop the few rows with NA values (only 190)
mqtt_raw = mqtt_raw.na.drop()
mqtt_test_raw = mqtt_test_raw.na.drop()

# Dropping a couple useless columns
mqtt_raw = mqtt_raw.drop("mqtt_protoname")
mqtt_test_raw = mqtt_test_raw.drop("mqtt_protoname")

In [ ]:
mqtt_raw.printSchema()

In [ ]:
col_names =  ["tcp_flags","tcp_time_delta","tcp_len",
"mqtt_conack_flags","mqtt_conack_flags_reserved","mqtt_conack_flags_sp",
"mqtt_conack_val","mqtt_conflag_cleansess","mqtt_conflag_passwd",
"mqtt_conflag_qos","mqtt_conflag_reserved","mqtt_conflag_retain",
"mqtt_conflag_uname","mqtt_conflag_willflag",
"mqtt_conflags","mqtt_dupflag","mqtt_hdrflags",
"mqtt_kalive","mqtt_len","mqtt_msg","mqtt_msgid","mqtt_msgtype",
"mqtt_proto_len","mqtt_qos",
"mqtt_retain","mqtt_sub_qos","mqtt_suback_qos","mqtt_ver",
"mqtt_willmsg","mqtt_willmsg_len","mqtt_willtopic","mqtt_willtopic_len",
"target"]

nominal_cols = ["mqtt_msgtype", "mqtt_conack_val", "tcp_flags", "mqtt_hdrflags"]
binary_cols = ["mqtt_dupflag"]
continuous_cols = ["tcp_time_delta","tcp_len",
"mqtt_conack_flags_reserved","mqtt_conack_flags_sp",
"mqtt_conflag_cleansess","mqtt_conflag_passwd",
"mqtt_conflag_qos","mqtt_conflag_reserved","mqtt_conflag_retain",
"mqtt_conflag_uname","mqtt_conflag_willflag",
"mqtt_kalive","mqtt_len","mqtt_msg","mqtt_msgid",
"mqtt_proto_len","mqtt_qos",
"mqtt_retain","mqtt_sub_qos","mqtt_suback_qos","mqtt_ver",
"mqtt_willmsg","mqtt_willmsg_len","mqtt_willtopic","mqtt_willtopic_len",
"mqtt_conack_flags", "mqtt_conflags"]

In [ ]:
# Add any columns that we want to just get rid of here
# A large portion of these were dropped because they were a constant value
columns_to_drop = ["mqtt_msg", "mqtt_conack_flags", "mqtt_conflags", 
"mqtt_willtopic_len", "mqtt_willtopic", "mqtt_willmsg_len", "mqtt_willmsg",
"mqtt_suback_qos", "mqtt_sub_qos", "mqtt_conflag_willflag", "mqtt_conflag_retain",
"mqtt_conflag_reserved", "mqtt_conflag_qos", "mqtt_conack_flags_sp", 
"mqtt_conack_flags_reserved"] 

In [ ]:
'''
Data preprocessing pipeline
'''
class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    def __init__(self):
        super().__init__()

    def label_to_vector(self, name):
        name = name.lower()
        
        if name == 'legitimate':
            return 0.0
        elif name == 'dos':
            return 1.0
        elif name == 'malformed':
            return 2.0
        elif name == 'slowite':
            return 3.0
        elif name == 'bruteforce':
            return 4.0
        elif name == 'flood':
            return 5.0
        else:
            return -100.0

    def _transform(self, dataset):
        label_to_vector_udf = udf(self.label_to_vector, StringType())
        output_df = dataset.withColumn('outcome', label_to_vector_udf(col('target'))).drop("target")  
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self, targetCols):
        super().__init__()
        self.target_cols = targetCols

    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.target_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df

class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    impute_columns = binary_cols+continuous_cols
    for col_name in columns_to_drop:
        if col_name in impute_columns:
            impute_columns.remove(col_name)
    stage_typecaster = FeatureTypeCaster(impute_columns)

    # Stage where coulmns are imputed if they have NAs
    stage_imputer = Imputer(inputCols=impute_columns, outputCols=impute_columns)

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols=nominal_cols, outputCols=nominal_id_cols)

    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = impute_columns+nominal_onehot_cols
    for col_name in columns_to_drop:
        if col_name in nominal_cols:
            feature_cols.remove(col_name+"_encoded")
            feature_cols.remove(col_name+"_index")
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columbs, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_cols + nominal_id_cols + nominal_onehot_cols +
        binary_cols + continuous_cols + columns_to_drop + ['vectorized_features'])
    # stage_column_dropper = ColumnDropper(columns_to_drop = nominal_id_cols + columns_to_drop + ['vectorized_features'])
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_typecaster, stage_imputer, stage_nominal_indexer, stage_nominal_onehot_encoder,
        stage_vector_assembler, stage_scaler, stage_outcome, stage_column_dropper])
    return pipeline

In [ ]:
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(mqtt_raw)

mqtt_df = preprocess_pipeline_model.transform(mqtt_raw)
mqtt_df_test = preprocess_pipeline_model.transform(mqtt_test_raw)

In [ ]:
mqtt_df_test.printSchema()

In [ ]:
mqtt_df_test.show(3, truncate=False, vertical=True)

<h1>Pyspark ML Models</h1>

In [ ]:
#Pyspark ml model 1 - Logistic Regression
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

lr = LogisticRegression(featuresCol = 'features', labelCol = 'outcome')
lrModel = lr.fit(mqtt_df) 

In [ ]:
#Pyspark ml model 2 - Random Forest Decision Tree
from pyspark.ml.classification import RandomForestClassifier

rfc = RandomForestClassifier(featuresCol = 'features', labelCol = 'outcome')
rf_model = rfc.fit(mqtt_df)

In [ ]:
pred_LR = lrModel.transform(mqtt_df_test)
y_pred = pred_LR.select('prediction').collect()
y = mqtt_df_test.select('outcome').collect()
correct = 0
for i in range(len(y)):
    if y[i][0]==y_pred[i][0]:
        correct += 1
print("Linear Regression Test Accuracy:",correct/len(y)*100, "%")

In [ ]:
pred_DT = rf_model.transform(mqtt_df_test)
y_pred_DT = pred_DT.select('prediction').collect()
correct = 0
for i in range(len(y)):
    if y[i][0]==y_pred_DT[i][0]:
        correct += 1
print("Decision Tree Test Accuracy:",correct/len(y)*100, "%")

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Hyperparameters for Linear Regression
lr_paramGrid = (ParamGridBuilder()
                .addGrid(lr.regParam, [0.005, 0.01, 0.05])
                .addGrid(lr.maxIter, [5, 10])
                .build())

# Hyperparameters for Random Forest
rf_paramGrid = (ParamGridBuilder()
                .addGrid(rfc.maxDepth, [5, 10])
                .addGrid(rfc.numTrees, [10, 20, 40])
                .build())

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',
                                        labelCol='outcome',
                                        metricName='accuracy')

lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid,
                       evaluator=evaluator, numFolds=5)
rf_cv = CrossValidator(estimator=rfc, estimatorParamMaps=rf_paramGrid,
                       evaluator=evaluator, numFolds=5)

In [ ]:
lr_cv_model = lr_cv.fit(mqtt_df)
lr_cv_prediction_test = lr_cv_model.transform(mqtt_df_test)
print('Linear Regression Hyperparameter tuning results: ')
print('Test Area under ROC (AUC) before Cross-Validation:', evaluator.evaluate(lr_test_predictions))
print('Test Area under ROC (AUC) after Cross-Validation:', evaluator.evaluate(lr_cv_prediction_test))
print()

rf_cv_model = rf_cv.fit(mqtt_df)
rf_cv_prediction_test = rf_cv_model.transform(mqtt_df_test)
print('Random Forest Hyperparameter tuning results: ')
print('Test Area under ROC (AUC) before Cross-Validation:', evaluator.evaluate(rfc_test_predictions))
print('Test Area under ROC (AUC) after Cross-Validation:', evaluator.evaluate(rf_cv_prediction_test))
print()

<h1>Pytorch Learning</h1>

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

import matplotlib.pyplot as plt

In [ ]:
mqtt_df_val, mqtt_df_test = mqtt_df_test.randomSplit([0.5, 0.5])

In [ ]:
mqtt_df_val_pd = mqtt_df_val.toPandas()
mqtt_df_train_pd = mqtt_df.toPandas()
mqtt_df_test_pd = mqtt_df_test.toPandas()

In [ ]:
x_train = torch.from_numpy(np.array(mqtt_df_train_pd['features'].values.tolist(),np.float32))
y_train = torch.from_numpy(np.array(mqtt_df_train_pd['outcome'].values.tolist(),np.int64))

x_test = torch.from_numpy(np.array(mqtt_df_test_pd['features'].values.tolist(),np.float32))
y_test = torch.from_numpy(np.array(mqtt_df_test_pd['outcome'].values.tolist(),np.int64))

x_val = torch.from_numpy(np.array(mqtt_df_val_pd['features'].values.tolist(),np.float32))
y_val = torch.from_numpy(np.array(mqtt_df_val_pd['outcome'].values.tolist(),np.int64))

In [ ]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self,idx):
        return(self.x[idx],self.y[idx])

train_dataset = MyDataset(x_train,y_train)
validate_dataset = MyDataset(x_val,y_val)
test_dataset = MyDataset(x_test,y_test)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size=2, output_size=1, hidden_layer_sizes=[24,24], activation_func = nn.ReLU):
        super().__init__()

        layers = []                                               # List of layers
        layers.append(nn.Linear(input_size,hidden_layer_sizes[0]))         # First hidden layer

                                                                  # Next hidden layers
        for i in range(len(hidden_layer_sizes)-1):
            layers.append(activation_func())
            layers.append(nn.Linear(hidden_layer_sizes[i],hidden_layer_sizes[i+1]))
            
        layers.append(activation_func())                                  # Output layer (if needed)
        layers.append(nn.Linear(hidden_layer_sizes[-1], output_size))
        
        self.seq = nn.Sequential(*layers)                         # Final sequential model

    def forward(self, xy):
        return self.seq(xy)

In [ ]:
def train(model, train_data, val_data, lr = 0.01, epochs = 1000, batch_size=32, gamma=1):
    # Define loss, optimizer, and scheduler
    lossfun = nn.functional.cross_entropy
    opt = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer=opt, gamma=gamma)

    # Define data loaders to do batch decent
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

    train_loss_hist = []
    val_loss_hist = []
    train_acc_hist = []
    val_acc_hist = []
    batch_loss_hist = []
    best_acc = 0.0
    for epoch in range(epochs+1):

        # Training batch loop for this epoch
        train_batch_loss=[]
        train_batch_acc=[]
        model.train()
        for _, (x_batch, y_batch) in enumerate(train_dataloader):
            test_batch_pred = model(x_batch)
            loss_train = lossfun(test_batch_pred, y_batch)
            
            opt.zero_grad()
            loss_train.backward()
            opt.step()

            train_batch_loss.append(loss_train.detach())
            test_batch_label = np.argmax(test_batch_pred.detach(), axis=1).numpy()
            train_batch_acc.append(np.sum(test_batch_label == y_batch.numpy())/y_batch.shape[0])

        # Validation batch loop for this epoch
        val_batch_loss = []
        val_batch_acc=[]
        model.eval()
        for _, (x_batch_val, y_batch_val) in enumerate(val_dataloader):
            val_batch_pred = model(x_batch_val)
            loss_val = lossfun(val_batch_pred, y_batch_val)

            val_batch_loss.append(loss_val.detach())
            val_batch_label = np.argmax(val_batch_pred.detach(), axis=1).numpy()
            val_batch_acc.append(np.sum(val_batch_label == y_batch_val.numpy())/y_batch_val.shape[0])

        scheduler.step()

        # Save data to histories
        batch_loss_hist += train_batch_loss
        train_loss_hist.append(np.mean(train_batch_loss))
        val_loss_hist.append(np.mean(val_batch_loss))
        train_acc_hist.append(np.mean(train_batch_acc))
        val_acc_hist.append(np.mean(val_batch_acc))

        # Print epoch info
        if epoch%int(epochs/10) == 0:
            print(f"Epoch {epoch:>4} of {epochs}:   Train Loss = {   train_loss_hist[-1]:.3f}   Validation Loss = {   val_loss_hist[-1]:.3f}"+
                                               f"   Train Acc  = {100*train_acc_hist[-1]:.3f}   Validation Acc  = {100*val_acc_hist[-1]:.3f}")

        # Save model if it is the best so far
        if val_acc_hist[-1] > best_acc:
            torch.save(model.state_dict(), 'current_best_model')
            best_acc = val_acc_hist[-1]

    plt.figure(figsize=(4,2),dpi=250)
    plt.plot(batch_loss_hist)
    plt.xlabel("Batch")
    plt.ylabel("Loss")
    plt.title("Loss across all batches")
    plt.show()

    fig, axs = plt.subplots(1,2)
    axs[0].plot(train_loss_hist,label="Training")
    axs[0].plot(val_loss_hist,label="Validation")
    axs[0].legend()
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].set_title("Loss across Epochs")

    axs[1].plot(train_acc_hist,label="Training")
    axs[1].plot(val_acc_hist,label="Validation")
    axs[1].legend()
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].set_title("Accuracy across Epochs")
    plt.show()

    return

In [ ]:
# Pytorch model #1 - Deep Nueral Network
hidden_layers = [128,128,128,128]
activation = nn.RReLU
net = NeuralNet(input_size=x_train.shape[1], output_size=6, hidden_layer_sizes=hidden_layers, activation_func=activation)
print(net)
train(net, train_dataset, validate_dataset, lr=0.005, epochs=20, batch_size=64, gamma=0.995)

In [ ]:
best_model = NeuralNet(input_size=x_train.shape[1], output_size=6, hidden_layer_sizes=hidden_layers, activation_func=activation)
best_model.load_state_dict(torch.load("current_best_model"))

prediction = np.argmax(best_model(x_test).detach(), axis=1).numpy()
accuracy = 100*np.sum(prediction == y_test.numpy())/y_test.shape[0]
print(f"Testing accuracy: {accuracy:.2f}%")

In [ ]:
# Pytorch model #2 - Shallow Nueral Network
hidden_layers = [8,8]
activation = nn.ReLU
net_shallow = NeuralNet(input_size=x_train.shape[1], output_size=6, hidden_layer_sizes=hidden_layers, activation_func=activation)
print(net_shallow)
train(net_shallow, train_dataset, validate_dataset, lr=0.05, epochs=10, batch_size=64, gamma=1)

In [ ]:
best_model_shallow = NeuralNet(input_size=x_train.shape[1], output_size=6, hidden_layer_sizes=hidden_layers, activation_func=activation)
best_model_shallow.load_state_dict(torch.load("current_best_model"))

prediction_shallow = np.argmax(best_model_shallow(x_test).detach(), axis=1).numpy()
accuracy_shallow = 100*np.sum(prediction_shallow == y_test.numpy())/y_test.shape[0]
print(f"Shallow network testing accuracy: {accuracy_shallow:.2f}%")